In [429]:
#pyvizenv environment needed
#commented out libraries & APIs we aren't currently using - kp

import requests
# initial imports
import os
import pandas as pd
import panel as pn
import plotly.express as px
#import plotly.graph_objects as go
import holoviews as hv
#from pathlib import Path
import datetime
from dotenv import load_dotenv
import json


load_dotenv()
rapid_api_key = os.getenv("RAPID_API_KEY")

In [430]:
import hvplot.pandas
#import matplotlib.pyplot as plt

In [431]:
# Read the Mapbox API key
#mapbox_token = os.getenv("MAPBOX")
#px.set_mapbox_access_token(mapbox_token)

In [432]:
#alpaca_token = os.getenv("ALPACA_API_KEY")

In [433]:

tickers_test_list = ["BTC-USD", "DJIA", "GLD", "DX-Y.NYB", "QQQ", "SPY", "WTI"]
list_of_stocks = []

def portfolio_builder():
    
    """
    Asks the user how many, and which stocks they'd like to choose for their portfolio.
    """
    
    #Clear list for new entries
    list_of_stocks.clear()
    
    #Initial Variables
    num_stocks='wrong'
    
    #Starts as string, must be turned into "digit".
    #int b4 input would break conditional
    while num_stocks.isdigit() == False:
    
        #Choosing a number
        num_stocks = input("How many stocks do you want in your portfolio?: ")
        
        #Digit Check
        if num_stocks.isdigit() == False:
            print("Sorry, that's not a digit!")
    
    #A Number has been chosen. Reflect that to the user.
    print(f"Your portfolio will have {num_stocks} stocks.\n")

    while len(list_of_stocks) < int(num_stocks):
        
        #What stocks are we adding?
        stocks = input("Which stocks would you like to choose?: ")
        
        #Did you put in a number?
        if stocks.isdigit() == True:
            print("Sorry, that's not a valid input!")
            
        #Did you already put that stock in the list?
        elif stocks in list_of_stocks:
            print("You've already chosen that stock. Please pick another!")
            
        #Seems legit. Let's add it to the list!
        else:
            list_of_stocks.append(stocks)
            
    #A list of stocks has been chosen. Reflect that to the user.    
    return print(f"Your portfolio will include {list_of_stocks}.\n")

In [434]:
# outputs textbox to key in stock symbol and uses api to get historical stock data for that stock
# use code to loop through list of tickers and output daily close data for x amount of time
# updated ticker list to be in same format - kp
# changed order in loop to print ticker symbol at top - kp
# updated url to make api data in json format, changed return to dump with indents - kp
def volume_grab(ticker):
    """
    Takes in arg "ticker" as a str object,
    and returns volume information for that
    symbol/ticker.
    
    date/open/high/low/close/volume/adjclose
    """
    
    #Inital Lists
    
    closing_date_list = []
    closing_volume_list = []
    
    #API Information
    
    url = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/stock/v3/get-historical-data"
    url += "?format=json"
    querystring = {"symbol":""}
    querystring['symbol']=ticker
    headers = {
        'x-rapidapi-host': "apidojo-yahoo-finance-v1.p.rapidapi.com",
        'x-rapidapi-key': rapid_api_key
        }
    response_data = requests.get(url, headers=headers, params=querystring)
    daily_data = response_data.json()
    daily_prices = daily_data['prices']
    
    for daily_info in daily_prices:
        try:
            closing_date_list.append(datetime.datetime.fromtimestamp(daily_info['date']).strftime('%Y-%m-%d'))
            closing_volume_list.append(daily_info['volume'])
        except:
            print(f" ")
            
    # Clean Date List incase of duplicates (stock splits etc.)
    
    clean_date_list = list(dict.fromkeys(closing_date_list))
    
    # Creating a Pandas DataFrame 
    
    d = {'Dates':clean_date_list, f"Volume {ticker}":closing_volume_list}
    df = pd.DataFrame(data=d)
    df.set_index('Dates', inplace=True)
    
    return df

def price_grab(ticker):
    """
    Takes in arg "ticker" as a str object,
    and returns price information for that
    symbol/ticker.
    
    date/open/high/low/close/volume/adjclose
    """
    
    #Inital Lists
    
    closing_price_list = []
    closing_date_list = []
    
    #API Information
    
    url = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/stock/v3/get-historical-data"
    url += "?format=json"
    querystring = {"symbol":""}
    querystring['symbol']=ticker
    headers = {
        'x-rapidapi-host': "apidojo-yahoo-finance-v1.p.rapidapi.com",
        'x-rapidapi-key': rapid_api_key
        }
    response_data = requests.get(url, headers=headers, params=querystring)
    daily_data = response_data.json()
    daily_prices = daily_data['prices']
    
    # For Loop grabbing API Data and appending lists
    
    for daily_info in daily_prices:
        try:
            closing_date_list.append(datetime.datetime.fromtimestamp(daily_info['date']).strftime('%Y-%m-%d'))
            closing_price_list.append(daily_info['close'])
        except:
            print(f" ")
    
    # Clean Date List incase of duplicates (stock splits etc.)
    
    clean_date_list = list(dict.fromkeys(closing_date_list))
    
    # Creating a Pandas DataFrame 
    
    d = {'Dates':clean_date_list, f"Closing Prices {ticker}":closing_price_list}
    df = pd.DataFrame(data=d)
    df.set_index('Dates', inplace=True)

    # Return/Display DataFrame
    
    return df

def pct_grab(ticker):
    
    """
    Takes in arg "ticker" as a str object,
    and returns pct_change information for that
    symbol/ticker.
    """
    
    #Inital Lists
    
    closing_price_list = []
    closing_date_list = []
    
    #API Information
    
    url = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/stock/v3/get-historical-data"
    url += "?format=json"
    querystring = {"symbol":""}
    querystring['symbol']=ticker
    headers = {
        'x-rapidapi-host': "apidojo-yahoo-finance-v1.p.rapidapi.com",
        'x-rapidapi-key': rapid_api_key
        }
    response_data = requests.get(url, headers=headers, params=querystring)
    daily_data = response_data.json()
    daily_prices = daily_data['prices']
    
    # For Loop grabbing API Data and appending lists
    
    for daily_info in daily_prices:
        try:
            closing_date_list.append(datetime.datetime.fromtimestamp(daily_info['date']).strftime('%Y-%m-%d'))
            closing_price_list.append(daily_info['close'])
        except:
            print(f" ")
    
    # Clean Date List incase of duplicates (stock splits etc.)
    
    clean_date_list = list(dict.fromkeys(closing_date_list))
    
    # Creating a Pandas DataFrame 
    
    d = {'Dates':clean_date_list, f"Closing Prices {ticker}":closing_price_list}
    df = pd.DataFrame(data=d)
    df.set_index('Dates', inplace=True)
    df[f'Pct Change {ticker}'] = df[f'Closing Prices {ticker}'].pct_change()
    df.drop(columns=f'Closing Prices {ticker}', inplace=True)

    # Return/Display DataFrame
    
    return df

#Test Code for JSON Return
#return(print(json.dumps(daily_data, indent=4)))

In [435]:
big_df_list = []

def create_price_df():
    """
    This function will create a price dataframe, given there
    are stocks in your portfolio. If not, try portfolio_builder()
    first.
    """
    
    big_df_list.clear()

    while len(big_df_list) != len(list_of_stocks):
        for x in list_of_stocks:
            big_df_list.append(price_grab(x))

    created_df = pd.concat(big_df_list, axis=1)
    created_df = created_df.iloc[::-1]
    
    return created_df

def create_volume_df():
    """
    This function will create a volume dataframe for your
    given portfolio. If not, try portfolio_builder() first.
    """
    
    big_df_list.clear()

    while len(big_df_list) != len(list_of_stocks):
        for x in list_of_stocks:
            big_df_list.append(volume_grab(x))

    created_df = pd.concat(big_df_list, axis=1)
    created_df = created_df.iloc[::-1]
    
    return created_df

def create_pctchg_df():
    """
    This function will create a volume dataframe for your
    given portfolio. If not, try portfolio_builder() first.
    """
    
    big_df_list.clear()

    while len(big_df_list) != len(list_of_stocks):
        for x in list_of_stocks:
            big_df_list.append(pct_grab(x))

    created_df = pd.concat(big_df_list, axis=1)
    created_df = created_df.iloc[::-1]
    
    return created_df

In [436]:
### PANELS NEED WORK, Y LIMITER ON VOLUME GRAPH..
### NEEDS REVISING

def create_panels():
    """
    Takes the three primary create functions
    and creates HV plot graphs.
    """
    
    # Creating the Graphs for the Panels
    change_graph = create_pctchg_df().hvplot(title='Percent Change', xlabel='Dates (One Year)',
                          ylabel='Percent Change',
                          width=1000, height=700)
    volume_graph = create_volume_df().hvplot(title='Volume', xlabel='Dates (One Year)',
                          yticks=(100000000,200000000,300000000),
                          ylabel='Volume (Hundred Millions)',
                         width=1000, height=700)
    price_graph = create_price_df().hvplot(title='Price', xlabel="Dates (One Year)",
                        ylabel='Price in USD', width=1000, height=700)
    
    # Setting up the Panels
    panel_one = pn.Column("##### Price", price_graph)
    panel_two = pn.Column("##### % Change", change_graph)
    panel_three = pn.Column("##### Volume", volume_graph)
    
    # Creating the tabs
    
    dashboard = pn.Tabs(("Prices", panel_one),
                        ("% Change", panel_two),
                        ("Volume",panel_three)
                       )
    
    return dashboard